In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from pathlib import Path
import time

from scipy.stats import ttest_rel

# sklearn
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score

from sklearn.base import BaseEstimator, TransformerMixin, clone

# Importações locais
# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_diab import *
from src.plot_metrica_class import *

print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
start_inicial = time.time()


#Processo iniciado em: 16:31:06


In [2]:
BASE = Path.cwd().parent   
# =====================================================
# ⚙️ 0. carregamento dos preprocessador 
# =====================================================
PP2 = joblib.load(BASE/'src'/'preprocess_diabetes_v1.4.joblib')['preprocessador']

# # =====================================================
# # 📁 1. Leitura dos dados & Separação das bases
# # =====================================================

DATA_DIR = BASE / "data" / "raw"
X_train = pd.read_csv(DATA_DIR / "X_train_raw.csv").reset_index(drop=True)
X_val  = pd.read_csv(DATA_DIR / "X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR / "y_train_raw.csv").values.ravel()
y_val  = pd.read_csv(DATA_DIR / "y_test_raw.csv")


# =====================================================
# 📁 2. Modelo e pipeline
# =====================================================
DATA_MODELS= BASE /"models"
pipe_LGBM1 = joblib.load(DATA_MODELS / 'modelo_LGBM_final_randsearch.roc_auc_v1.4.joblib')
pipe_LGBM1.named_steps # informações sobre a pipeline

{'preprocess': ColumnTransformer(transformers=[('num',
                                  Pipeline(steps=[('imputer',
                                                   SimpleImputer(strategy='median')),
                                                  ('cast_float32',
                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                       func=<function to_float32 at 0x7608711c8c10>))]),
                                  Index(['age', 'alcohol_consumption_per_week', 'bmi', 'cholesterol_total',
        'diastolic_bp', 'diet_score', 'hdl_cholesterol', 'heart_rate',
        'ldl_cho...
                                  Index(['gender', 'ethnicity', 'education_level', 'income_level',
        'smoking_status', 'employment_status'],
       dtype='object')),
                                 ('bin',
                                  Pipeline(steps=[('cast_int8',
                              

In [3]:
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))

# # =====================================================
# # 3.Treino & Teste
# # =====================================================
pipe_LGBM1.fit(X_train, y_train)
y_pred=pipe_LGBM1.predict(X_val) #validação


print(f"{'='*70}")
print(f"🎯 LGBM ")
print(f"{'='*70}")
print(f"📊 **Acurácia no Teste**: {accuracy_score(y_val, y_pred):.4f}")
print(f"\n📋 **Relatório de Classificação**:")
print(classification_report(y_val, y_pred))
cm=confusion_matrix(y_val, y_pred)
print(f"🎯 **Matriz de Confusão**:")
print(f"               Previsto 0   Previsto 1")
print(f"Real 0         {cm[0,0]:<11} {cm[0,1]:<11}")
print(f"Real 1         {cm[1,0]:<11} {cm[1,1]:<11}")
print(f"{'─'*70}")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))



#Processo iniciado em: 16:31:07
🎯 LGBM 
📊 **Acurácia no Teste**: 0.6862

📋 **Relatório de Classificação**:
              precision    recall  f1-score   support

         0.0       0.62      0.43      0.51     79108
         1.0       0.71      0.84      0.77    130892

    accuracy                           0.69    210000
   macro avg       0.66      0.64      0.64    210000
weighted avg       0.68      0.69      0.67    210000

🎯 **Matriz de Confusão**:
               Previsto 0   Previsto 1
Real 0         34120       44988      
Real 1         20913       109979     
──────────────────────────────────────────────────────────────────────

#Processo finalizado em: 16:31:17


In [4]:
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))

# =====================================================
# Submissão Kaggle
# =====================================================
DATA_DIR = BASE / "data" / "raw"
base = pd.read_csv(DATA_DIR / "test.csv")

id_test = base["id"]

x_test=base.drop(columns='id')

# Previsão
y_pred=pipe_LGBM1.predict(x_test)

# y_probs = pipe_RF3.predict_proba(x_test)[:, 1]
# y_pred = (y_probs >= best_threshold).astype(int)

#  DataFrame de submissão
submission = pd.DataFrame({
    "id": id_test,
    "Survived": y_pred
})
submission_path = ("/home/akel/PycharmProjects/Kaggle/Diabetes_Prediction_Challenge/data/process/submission_LGBM_final_randsearch.roc_auc_v14.csv")
submission.to_csv(submission_path, index=False)
# print("✅ Arquivo de submissão salvo com sucesso!")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo iniciado em: 16:31:17

#Processo finalizado em: 16:31:19
